In [1]:
# default_exp cache

# Cache

Le but du cache est de mémoriser des résultats pour éviter de les recalculer.

Nous utilisons http://redis.com

Pour l'utiliser en local :

In [2]:
#!docker run -d -p 6379:6379 --name redis redis:alpine

Pour l'arrêter :
 `docker stop redis`
 

In [3]:
#!docker stop redis

Pour le redémarrer :

In [4]:
!docker start redis

/bin/bash: docker: command not found


In [5]:
# export

import unittest

import redis
from redis import exceptions as redisExceptions

from leximpact_socio_fisca_simu_etat.config import Configuration
from leximpact_socio_fisca_simu_etat.logger import logger

tc = unittest.TestCase()

In [6]:
# export


class Cache:

    cache = None
    abuse_script = None
    # LUA script for API throttling
    # Thanks to https://github.com/long2ice/fastapi-limiter/blob/master/fastapi_limiter/__init__.py
    lua_abuse_script = """
        local key = KEYS[1]
        local limit = tonumber(ARGV[1])
        local expire_time = ARGV[2]
        local current = tonumber(redis.call('get', key) or "0")
        if current > 0 then
            if current + 1 > limit then
                return redis.call("PTTL",key)
            else
                redis.call("INCR", key)
                return 0
            end
        else
            redis.call("SET", key, 1,"px",expire_time)
            return 0
        end"""

    def __init__(self):
        config = config = Configuration()
        redis_host = config.get("REDIS_HOST", fail_on_missing=False)
        redis_port = config.get("REDIS_PORT", fail_on_missing=False, default=6379)
        redis_password = config.get("REDIS_PASSWORD", fail_on_missing=False)
        try:
            logger.debug(f"Connecting to Redis {redis_host}:{redis_port}")
            self.cache = redis.Redis(
                host=redis_host, password=redis_password, port=redis_port
            )
            if self.cache:
                self.cache.set("init", "OK")
                self.abuse_script = self.cache.register_script(self.lua_abuse_script)
        except redisExceptions.ConnectionError as e:
            self.cache = None
            logger.warning(f"Unable to connect to Redis cache : {str(e)}")

    def is_available(self):
        try:
            if self.get("init") == b"OK":
                return True
            else:
                return False
        except redisExceptions.ConnectionError as e:
            self.cache = None
            logger.warning(f"Unable to connect to Redis cache : {str(e)}")
            return False

    def clear_cache(self):
        # Flush the Redis DB
        if self.cache is None:
            logger.warning("Cache().clear_cache() : No Redis connection.")
            return False
        status = self.cache.flushdb()
        if status:
            # Set init as it can be use by API to now if cache works
            self.cache.set("init", "OK")
            logger.debug("Redis cache cleared successfully.")
            return True
        else:
            logger.warning("Unable to clear Redis cache.")
            return False

    def set(self, key, value):
        if self.cache is None:
            logger.warning("Cache().set(key, value) : No Redis connection.")
            return False
        try:
            return self.cache.set(key, value)
        except redisExceptions.ConnectionError as e:
            logger.warning(f"Unable to use Redis cache : {str(e)}")
            return False

    def get(self, key):
        if self.cache is None:
            # logger.warning("Cache().get(key) : No Redis connection.")
            return None
        try:
            return self.cache.get(key)
        except redisExceptions.ConnectionError as e:
            logger.warning(f"Unable to use Redis cache : {str(e)}")
            return None

    def is_abusing(self, identifier, nb_call_per_minute=5):
        if self.abuse_script is None:
            logger.warning("Unable to use Redis cache to check for abuse !")
            return False
        pexpire = self.abuse_script(
            keys=[identifier],
            args=[str(nb_call_per_minute), str(60 * 1000)],
            client=self.cache,
        )
        # The script return the time to wait before beeing allowed to call again
        # 0 if allowed
        if pexpire == 0:
            return False
        return True

Insertion d'une valeur dans le cache

In [7]:
cache = Cache()
cache.set("clef", "valeur")
print(cache.get("init"))

[leximpact_socio-fisca-simu-etat DEBUG @ 08:21:47] Connecting to Redis 10.0.0.131:6377


b'OK'


In [8]:
if cache.get("init") == b"OK":
    print("OUI")
else:
    print("NON")

OUI


Récupération d'une valeur

In [9]:
print(cache.get("clef"))

b'valeur'


In [10]:
tc.assertEqual(cache.get("clef"), b"valeur")

Effacement **complet** de **tout** le cache, pour **tous** les utilisateurs.

In [11]:
cache.clear_cache()

[leximpact_socio-fisca-simu-etat DEBUG @ 08:21:47] Redis cache cleared successfully.


True

In [12]:
assert cache.get("clef") == None

In [13]:
res = cache.get("5078a86c7201f132a44472774283e4a774e85b9bd94c88c9e756d3cb2021")
if res is None:
    print("N'existe pas")
else:
    print(res)

N'existe pas


### Rate limiting

In [14]:
i = 0
key = "user_key2"
while (not cache.is_abusing(key)) and i < 10:
    # print(cache.get(key))
    i += 1
print(cache.cache.pttl(key))
print(cache.get(key))
print(f"Abusing after {i} call")
tc.assertEqual(i, 5)

60000
b'5'
Abusing after 5 call


## Utilisation

Import du module

In [15]:
from leximpact_socio_fisca_simu_etat.cache import Cache

cache.set("clef", "valeur")
print(cache.get("clef"))

b'valeur'
